### Question search engine with BERT-like model

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.10

In [ ]:
import numpy as np

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
qqp['train'][0]

{'text1': 'How is the life of a math student? Could you describe your own experiences?',
 'text2': 'Which level of prepration is enough for the exam jlpt5?',
 'label': 0,
 'idx': 0,
 'label_text': 'not duplicate'}

In [ ]:
# qqp_preprocessed['train'][0]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Evaluation on QQP task


In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=64, shuffle=False, collate_fn=transformers.default_data_collator,
    num_workers=2
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sample batch: {'labels': tensor([0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0]), 'idx': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63]), 'input_ids': tensor([[ 101, 2009, 1132,  ...,    0,    0,    0],
        [ 101,  146, 1328,  ...,    0,    0,    0],
        [ 101, 2181, 1175,  ...,    0,    0,    0],
        ...,
        [ 101, 1327, 1209,  ...,    0,    0,    0],
        [ 101, 1327,  112,  ...,    0,    0,    0],
        [ 101, 1327, 1110,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],


In [ ]:
device = 'cuda'
model.to(device)
N = 0
TP = 0

for batch in val_loader:
  with torch.no_grad():
    predicted = model(
      input_ids=batch['input_ids'].to(device),
      attention_mask=batch['attention_mask'].to(device),
      token_type_ids=batch['token_type_ids'].to(device)
    )
    y_true = batch['labels'].data.numpy()
    # y_pred = np.argmax(torch.softmax(predicted.logits, dim=1).cpu().data.numpy())
    y_pred = np.array([np.argmax(el) for el in torch.softmax(predicted.logits, dim=1).cpu().data.numpy()])
    N += len(y_true)
    TP += np.sum(y_true == y_pred)


accuracy = TP / N


In [ ]:
accuracy

0.9083848627256987

### Training deberta

In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
model_name = 'microsoft/deberta-v3-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model

In [ ]:
train_loader = torch.utils.data.DataLoader(qqp_preprocessed['train'],
                                           batch_size=8,
                                           shuffle=True,
                                           num_workers=2,
                                           collate_fn=transformers.default_data_collator)

In [ ]:
model = model.to(device)

In [ ]:
model.parameters()

<generator object Module.parameters at 0x7c275ecc7bc0>

In [ ]:
for name, param in model.named_parameters():
  if name != 'classifier.bias':
    param.requires_grad = False

In [ ]:
list(model.named_parameters())[-1]

('classifier.bias',
 Parameter containing:
 tensor([0., 0.], device='cuda:0', requires_grad=True))

In [ ]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
qqp_preprocessed['train']

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 363846
})

In [ ]:
from tqdm import tqdm

In [ ]:
##### TRAINING MODEL
num_epochs = 5
n_iter = 150

for epoch in range(num_epochs):
  cur_iter = 0
  TP = 0
  N = 0

  ##### train
  model.train(True)
  for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
    out = model(input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device))

    y_true = batch['labels'].long()

    loss = loss_fn(out.logits.cpu(), y_true)

    loss.backward()

    optimizer.step()

    optimizer.zero_grad()

    cur_iter += 1
    if cur_iter > n_iter:
      break

  ##### validation
  cur_iter = 0

  model.train(False)
  for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch"):
    out = model(input_ids=batch['input_ids'].to(device),
                      attention_mask=batch['attention_mask'].to(device),
                      token_type_ids=batch['token_type_ids'].to(device))

    y_pred = np.array([np.argmax(el) for el in torch.softmax(out.logits, dim=1).cpu().data.numpy()])
    y_true = batch['labels'].data.numpy()

    N += len(y_true)
    TP += np.sum(y_true == y_pred)

    cur_iter += 1
    if cur_iter > n_iter:
      break

  print(f'Epoch: {epoch + 1}, valid. accuracy: {TP / N}')


Epoch 1/5:   0%|          | 150/45481 [00:11<56:33, 13.36batch/s]


Epoch: 1, valid. accuracy: 0.6167218543046358


Epoch 2/5:   0%|          | 150/45481 [00:11<57:41, 13.09batch/s]


Epoch: 2, valid. accuracy: 0.6307947019867549


Epoch 3/5:   0%|          | 150/45481 [00:11<58:10, 12.99batch/s]


Epoch: 3, valid. accuracy: 0.6200331125827815


Epoch 4/5:   0%|          | 150/45481 [00:11<57:45, 13.08batch/s]


Epoch: 4, valid. accuracy: 0.6225165562913907


Epoch 5/5:   0%|          | 150/45481 [00:11<57:57, 13.04batch/s]

Epoch: 5, valid. accuracy: 0.6415562913907285


In [ ]:
# qqp_preprocessed['train']

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 363846
})

In [ ]:
# qqp_preprocessed['train'][0]['token_type_ids'][0:18]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
print(len(qqp_preprocessed['train'][0]['attention_mask']))
print(len(qqp_preprocessed['train'][0]['token_type_ids']))
print(len(qqp_preprocessed['train'][0]['text1']))
print(len(qqp_preprocessed['train'][0]['text2']))

128
128
75
55


In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    result['text1'] = examples['text1']
    result['text2'] = examples['text2']
    return result


In [ ]:
# qqp_preprocessed['train'][0]

In [ ]:
train_loader = torch.utils.data.DataLoader(qqp_preprocessed['train'],
                                           batch_size=1,
                                           shuffle=False,
                                           num_workers=1,
                                           collate_fn=transformers.default_data_collator)

In [ ]:
def create_custom_loader(question: str):
  new_dataset = []

  for i in tqdm(range(10000)):
    obj = qqp['train'][i]
    text1 = obj['text1']
    text2 = obj['text2']

    variant_1 = {
        'text1': question,
        'text2': text2,
        'label': None,
        'idx': None,
        'label_text': None,
    }
    variant_2 = {
        'text1': text1,
        'text2': question,
        'label': None,
        'idx': None,
        'label_text': '',
    }

    var_1_preprocessed = preprocess_function(variant_1)
    var_2_preprocessed = preprocess_function(variant_2)

    new_dataset.append(var_1_preprocessed)
    new_dataset.append(var_2_preprocessed)

  return new_dataset

In [ ]:
part_train = create_custom_loader('What can you say about upcoming USA election?')

100%|██████████| 10000/10000 [00:11<00:00, 895.91it/s]


In [ ]:
# part_train = np.array(part_train)

In [ ]:
# part_train

In [ ]:
'''
Copied from official documentaion (transformers.default_data_collator),
than modified for needs.
'''

from collections.abc import Mapping
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union


InputDataClass = NewType("InputDataClass", Any)

def modified_data_collator(features: List[InputDataClass]) -> Dict[str, Any]:
    import torch

    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if isinstance(first["label_ids"][0], int) else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            elif isinstance(v, np.ndarray):
                batch[k] = torch.tensor(np.stack([f[k] for f in features]))
            else:
                batch[k] = torch.tensor([f[k] for f in features])

        ### BEGIN: modified part
        else:
          if k in ('text1', 'text2'):
            batch[k] = [f[k] for f in features][0]
        ### END: modified part

    return batch

In [ ]:
train_loader = torch.utils.data.DataLoader(part_train,
                                           batch_size=1,
                                           shuffle=False,
                                           num_workers=1,
                                           collate_fn=modified_data_collator,
                                          #  collate_fn=transformers.default_data_collator,
                                           )

In [ ]:
for obj in train_loader:
  print(obj)
  break

{'input_ids': tensor([[  101,  1327,  1169,  1128,  1474,  1164,  8851,  3066,  1728,   136,
           102,  5979,  1634,  1104,  3073, 20488,  2116,  1110,  1536,  1111,
          1103, 12211,   179,  1233,  6451,  1571,   136,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
device='cuda'
model = model.to(device)

def find_duplicate(question: str):
  most_similar = []
  for obj in tqdm(train_loader):

    out = model(input_ids=obj['input_ids'].to(device),
                attention_mask=obj['attention_mask'].to(device),
                token_type_ids=obj['token_type_ids'].to(device))

    score = torch.softmax(out.logits, dim=1).cpu().data.numpy()[0][1]

    flag_of_text = 0 if obj['text1'] == question else 1
    candidate = obj['text1'] if flag_of_text else obj['text2']
    pair = (score, candidate)
    most_similar.append(pair)
  return sorted(most_similar, reverse=True)

find_duplicate('What can you say about upcoming USA election?')[:10]

100%|██████████| 20000/20000 [04:55<00:00, 67.75it/s]


[(0.9998683, "What's your resolutions for 2017?"),
 (0.99985933, 'What should be strategy for CAT 2017?'),
 (0.9998332,
  'What is your reaction to Donald Trump winning the 2016 US presidential election?'),
 (0.999821, 'What is your New Year Resolution for 2017?'),
 (0.999821, 'What is your New Year Resolution for 2017?'),
 (0.999821, 'What is your New Year Resolution for 2017?'),
 (0.9998091, 'What are your new year resolutions’2017?'),
 (0.9997187, 'What is the expected cut off for KVPY SA 2016?'),
 (0.999708, 'What Is your New year resolutions in 2017?'),
 (0.999708, 'What Is your New year resolutions in 2017?')]